In [1]:
import pandas as pd
import numpy as np
from statsmodels.stats import weightstats as ws

In [2]:
df_raw = pd.read_csv('test_2.csv')
df_raw

,u_id,is_international,is_old_client,ab_group,payment_approved
0,79be70c3e6216e036f2fcd10d8a61441,0,0,last-seats,0
1,1a35c1aad504fb0007f12192c1a61bdf,0,0,control,0
2,a1eef4b38764276ca72200a16f383020,0,0,last-seats,0
3,a423207fd6f13312e63fde3b6068f1bf,0,1,last-seats,0
4,b0f0d61e8ad381693581893f24e0fdfe,0,0,control,0
...,...,...,...,...,...
82402,dcecb5c0311115302465fb05debaecff,1,0,control,0
82403,466ae995def313e246ef7772963b8fa4,1,0,last-seats,0
82404,f059408efc41fb35bb87a648e40793b3,1,1,last-seats,1
82405,88d4261aed7a76ea8cfb8cbd7f927eab,1,0,control,0


In [3]:
#Смотрим статзначимость в тотале
df_ab = df_raw.groupby(['u_id', 'ab_group'], as_index=False).agg({'payment_approved':'max'})

m_1 = df_ab.loc[df_ab['ab_group'] == 'control', 'payment_approved'].mean()
m_2 = df_ab.loc[df_ab['ab_group'] == 'last-seats', 'payment_approved'].mean()

delta = m_2 - m_1
delta_perc = (m_2/m_1-1)*100

p_value = ws.ttest_ind(
    np.array(df_ab.loc[df_ab['ab_group'] == 'control', 'payment_approved']),
    np.array(df_ab.loc[df_ab['ab_group'] == 'last-seats', 'payment_approved']),
    alternative='two-sided',
    usevar='unequal'
)[1]

df_result = pd.DataFrame([['CR to payment success', 'all', m_1, m_2, delta, delta_perc, p_value]], columns=['metric', 'segment', 'mean control', 'mean test', 'delta total', 'delta_perc', 'p_value'])
df_result

,metric,segment,mean control,mean test,delta total,delta_perc,p_value
0,CR to payment success,all,0.097479,0.103288,0.005809,5.959571,0.008354


In [4]:
#Проверяем различие внутренних и международных рейсов
df_segment = df_raw.groupby(['u_id', 'ab_group', 'is_international'], as_index=False).agg({'payment_approved':'max'})

df_result = pd.DataFrame()
for i in [0, 1]:
    m_1 = df_segment.loc[(df_segment['ab_group'] == 'control') & (df_segment['is_international'] == i), 'payment_approved'].mean()
    m_2 = df_segment.loc[(df_segment['ab_group'] == 'last-seats') & (df_segment['is_international'] == i), 'payment_approved'].mean()

    delta = m_2 - m_1
    delta_perc = (m_2/m_1-1)*100

    df_temp = pd.DataFrame([['CR to payment success', i, m_1, m_2, delta, delta_perc]], columns=['metric', 'segment', 'mean control', 'mean test', 'delta total', 'delta_perc'])
    df_result = pd.concat([df_result, df_temp], axis=0)
df_result

,metric,segment,mean control,mean test,delta total,delta_perc
0,CR to payment success,0,0.104413,0.112469,0.008056,7.715688
0,CR to payment success,1,0.056611,0.055630,-0.000981,-1.733292


In [5]:
df_segment = df_raw.groupby(['u_id', 'ab_group', 'is_old_client'], as_index=False).agg({'payment_approved':'max'})

df_result = pd.DataFrame()
for i in [0, 1]:
    m_1 = df_segment.loc[(df_segment['ab_group'] == 'control') & (df_segment['is_old_client'] == i), 'payment_approved'].mean()
    m_2 = df_segment.loc[(df_segment['ab_group'] == 'last-seats') & (df_segment['is_old_client'] == i), 'payment_approved'].mean()

    delta = m_2 - m_1
    delta_perc = (m_2/m_1-1)*100

    df_temp = pd.DataFrame([['CR to payment success', i, m_1, m_2, delta, delta_perc]], columns=['metric', 'segment', 'mean control', 'mean test', 'delta total', 'delta_perc'])
    df_result = pd.concat([df_result, df_temp], axis=0)
df_result

,metric,segment,mean control,mean test,delta total,delta_perc
0,CR to payment success,0,0.089483,0.096379,0.006896,7.706627
0,CR to payment success,1,0.113253,0.117011,0.003758,3.318445
